In [10]:
#!pip install SpeechRecognition
!pip install pyaudio

     -------------------------------------- 164.1/164.1 kB 5.0 MB/s eta 0:00:00


In [67]:
import speech_recognition as sr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import pyaudio
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\VBALA\AppData\Roaming\nltk_data...


True

In [19]:
def main():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("Speak now..")
        audio = r.listen(source)
        try:
            out=r.recognize_google(audio)
            return out
        except Exception as e:
            return "Error: "+str(e)
text=main()
print(text)

Speak now..
Error: 


In [56]:
column_names = ['ID', 'Product', 'Sentiment', 'Text']
df = pd.read_csv('C:/Users/VBALA/Downloads/Projects/twitter_training.csv')

In [57]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [58]:
np.unique(df['Borderlands'])

array(['Amazon', 'ApexLegends', 'AssassinsCreed', 'Battlefield',
       'Borderlands', 'CS-GO', 'CallOfDuty', 'CallOfDutyBlackopsColdWar',
       'Cyberpunk2077', 'Dota2', 'FIFA', 'Facebook', 'Fortnite', 'Google',
       'GrandTheftAuto(GTA)', 'Hearthstone', 'HomeDepot',
       'LeagueOfLegends', 'MaddenNFL', 'Microsoft', 'NBA2K', 'Nvidia',
       'Overwatch', 'PlayStation5(PS5)',
       'PlayerUnknownsBattlegrounds(PUBG)', 'RedDeadRedemption(RDR)',
       'TomClancysGhostRecon', 'TomClancysRainbowSix', 'Verizon',
       'WorldOfCraft', 'Xbox(Xseries)', 'johnson&johnson'], dtype=object)

In [59]:
np.unique(df['Positive'])

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

In [60]:
df = df.drop('2401' , axis=1)

In [61]:
df= df.rename(columns={"Borderlands":"Feature2","im getting on borderlands and i will murder you all ,":"Feature1","Positive": "labels"})

In [62]:
df.head()

,Feature2,labels,Feature1
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...


In [72]:
del df['labels']
df['Feature1'] = df['Feature1'].str.lower()

In [73]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    # Check if the input is a float and convert it to a string
    if isinstance(text, float):
        text = str(text)
    
    sentiment = sia.polarity_scores(text)
    
    if sentiment['compound'] >= 0.05:
        return 'Positive'
    elif sentiment['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Apply the updated get_sentiment function to the DataFrame
df['NLTK_Sentiment'] = df['Feature1'].apply(get_sentiment)


In [74]:
df.head()

,Feature2,Feature1,NLTK_Sentiment
0,Borderlands,i am coming to the borders and i will kill you...,Negative
1,Borderlands,im getting on borderlands and i will kill you ...,Negative
2,Borderlands,im coming on borderlands and i will murder you...,Negative
3,Borderlands,im getting on borderlands 2 and i will murder ...,Negative
4,Borderlands,im getting into borderlands and i can murder y...,Negative


In [75]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')

sentiment_classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

def classify_sentiment(text):
    sentiment_result = sentiment_classifier(text)
    return sentiment_result[0]['label']

df['RoBERTa_Sentiment'] = df['Feature1'].apply(classify_sentiment)


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

C:\Users\VBALA\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VBALA\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

KeyboardInterrupt: 

In [76]:
df.head()

,Feature2,Feature1,NLTK_Sentiment
0,Borderlands,i am coming to the borders and i will kill you...,Negative
1,Borderlands,im getting on borderlands and i will kill you ...,Negative
2,Borderlands,im coming on borderlands and i will murder you...,Negative
3,Borderlands,im getting on borderlands 2 and i will murder ...,Negative
4,Borderlands,im getting into borderlands and i can murder y...,Negative


In [77]:
get_sentiment('I am lonely, don\'t know what to do')

'Negative'

In [ ]:
get_sentiment('such ')